# Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
Training_data_file='SpamData/02_Training/train_data.txt'
Testing_data_file='SpamData/03_Testing/test_data.txt'

Token_spam_prob='SpamData/03_Testing/prob_spam.txt'
Token_ham_prob='SpamData/03_Testing/prob_ham.txt'
Token_all_prob='SpamData/03_Testing/prob_all.txt'

Test_feature_mtrx='SpamData/03_Testing/test_features.txt'
Test_target_file='SpamData/03_Testing/test_target.txt'

# Read features

In [3]:
sparse_train_data = np.loadtxt(Training_data_file, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(Testing_data_file, delimiter=' ', dtype=int)

In [5]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, 2500))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [6]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [7]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, 2500))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [8]:
%%time
full_train_data = make_full_matrix(sparse_train_data, 2500)

Wall time: 12 s


In [9]:
sz=full_train_data.CATEGORY.size
print(sz)

4013


In [10]:
spam_brk=full_train_data.query("CATEGORY == 1").shape[0]

In [11]:
hg=(full_train_data.CATEGORY==1).sum()
hg

1248

In [12]:
prob_spam=spam_brk/sz

In [13]:
prob_spam

0.310989284824321

# nbr of words

In [14]:
full_train_features=full_train_data.loc[:, full_train_data.columns!='CATEGORY']

In [15]:
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
email_lengths=full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [17]:
total_wc=email_lengths.sum()

In [18]:
total_wc

429249

 # nbr of token in spam et ham

In [19]:
spam_length=email_lengths[full_train_data.CATEGORY==1]

In [20]:
spam_length.shape

(1248,)

In [21]:
spam_wc=spam_length.sum()

In [22]:
spam_wc

176324

In [23]:
ham_length=email_lengths[full_train_data.CATEGORY==2]
ham_length.shape

(2765,)

In [24]:
email_lengths.shape[0]-spam_length.shape[0]-ham_length.shape[0]

0

In [25]:
ham_wc=ham_length.sum()

In [26]:
ham_wc

252925

# sum of tokens

In [27]:
train_spam_tokens=full_train_features.loc[full_train_data.CATEGORY == 1]

In [28]:
train_spam_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens

0       2179
1        935
2       1217
3       2022
4       1219
        ... 
2495       3
2496       1
2497       1
2498      21
2499       4
Length: 2500, dtype: int64

In [30]:
train_ham_tokens=full_train_features.loc[full_train_data.CATEGORY==2]

In [31]:
summed_ham_tokens=train_ham_tokens.sum(axis=0)+1
summed_ham_tokens

0       5484
1       2590
2       2045
3        938
4       1612
        ... 
2495      27
2496      33
2497      25
2498      10
2499      36
Length: 2500, dtype: int64

In [32]:
summed_ham_tokens[2496].sum()

33

In [33]:
train_ham_tokens[2496
                ].sum()

32

 # p(token | spam)

In [34]:
prob_tokens_spam=summed_spam_tokens/ (spam_wc + 2500)
prob_tokens_spam

0       0.012185
1       0.005229
2       0.006806
3       0.011307
4       0.006817
          ...   
2495    0.000017
2496    0.000006
2497    0.000006
2498    0.000117
2499    0.000022
Length: 2500, dtype: float64

In [35]:
prob_tokens_spam.sum()

1.0

 # p(token | ham)

In [36]:
prob_tokens_ham=summed_ham_tokens /(ham_wc + 2500)

In [37]:
prob_tokens_ham.sum()

1.0

# p(token)

In [38]:
prob_token=full_train_features.sum(axis=0)/total_wc

In [39]:
prob_token.sum()

1.0

# save files

In [40]:
np.savetxt(Token_spam_prob,prob_tokens_spam)
np.savetxt(Token_ham_prob,prob_tokens_ham)
np.savetxt(Token_all_prob,prob_token)

# prepare test data


In [41]:
full_test_data=make_full_matrix(sparse_test_data,nr_words=2500)

In [42]:
X_test=full_test_data.loc[:,full_test_data.columns!='CATEGORY']
y_test=full_test_data.CATEGORY

In [43]:
np.savetxt(Test_target_file,y_test)
np.savetxt(Test_feature_mtrx,X_test)

In [44]:
y_test[:5]

DOC_ID
8     1
12    1
14    1
15    1
17    1
Name: CATEGORY, dtype: int64